In [1]:
import json
from collections import defaultdict


In [2]:
with open("datasets_study_final.json", "r") as f:
    data = json.load(f) 

analysis_data = []
prompt_data = []    

dataset_name = defaultdict(list)
for item in data:
    if item['model']=='datasets_study.analysis':
        analysis_data.append(item)
    elif item['model']=='datasets_study.prompt':
        prompt_data.append(item)
        dataset_name[item['fields']['source_dataset']].append(item['pk'])




len(analysis_data),len(prompt_data)
print(len(dataset_name.keys()))

20


In [3]:
problem_list = {}
for item in data:
    if item['model']=='datasets_study.problem':
       problem_list[item['pk']] = item['fields']['description']
print(problem_list)

{1: 'Spelling and grammatical errors', 2: 'Very short sentence', 3: 'Partial or incomplete sentence', 4: 'Self-Admitted Technical Debt', 5: 'Automatically generated code/comments', 6: "Method name mismatches with the function/method's intent.", 7: 'Not using JavaDoc (Java) or docstring (Python) on the prompt.', 8: 'Inconsistent prompt style', 9: 'URL or reference in the comment', 12: 'Interrogation questions in the prompt', 13: 'JavaDoc/docstring has formatting issues', 14: 'Comment includes PII(author name, email address, etc.)', 15: 'The prompt description is unclear', 16: 'Incorrect input/output pair example'}


In [4]:
users = defaultdict(list)
analysis = defaultdict(list)
problems = defaultdict(list)
dataset_data = defaultdict(list)

problem_ids = defaultdict(list)


for item in analysis_data:
    # if item['fields']['user_id'] != 4:
    #     continue
    
    users[item['fields']['user_id']].append(item['fields'])
    analysis[item['fields']['prompt']].append(item['fields'])
    for problem in item['fields']['problems']:

        problems[problem].append(item['fields'])
        if problem == 13:
            problem = 7
        problem_ids[problem].append(item['fields']['prompt'])

    for data in dataset_name.keys():
        if item['fields']['prompt'] in dataset_name[data]:
            dataset_data[data].append(item['fields'])
            break
print(users.keys())
print(len(analysis))
print(len(problems))    

total = 0
for data in dataset_data.keys():
    # print(data, len(dataset_data[data]))
    if len(dataset_name[data]) == len(dataset_data[data]):
        print(data, len(dataset_name[data]), len(dataset_data[data]))


dict_keys([4, 3, 1, 11])
4164
12
HumanEval-Infilling/HumanEval-MultiLineInfilling 598 598


In [6]:
problem_ids[16]

[12,
 84,
 123,
 1130,
 1159,
 1167,
 402,
 536,
 576,
 582,
 594,
 638,
 648,
 673,
 687,
 693,
 708,
 721,
 728,
 729,
 750,
 227,
 240,
 273,
 303,
 351,
 771,
 778,
 779,
 792,
 796,
 812,
 823,
 835,
 839,
 854,
 865,
 898,
 906,
 924,
 925,
 936,
 12,
 123,
 227,
 240,
 273,
 351,
 402,
 536,
 576,
 638,
 925,
 936,
 1064,
 1145,
 1159]

In [ ]:
problems = sorted(problems.items(), key=lambda x: x[0])
# print(problems)

In [ ]:
for problem in problem_ids:
    print(problem, len(problem_ids[problem]), len(set(problem_ids[problem])))

In [ ]:
for item in problems:
    key = item[0]
    total = len(item[1])
    print(f"{key}. {problem_list[key]}: {total}")


In [ ]:
merged_analysis = defaultdict(list)
for item in analysis_data:
    if item['fields']['user_id'] != 3:
        continue
    merged_analysis[item['fields']['prompt']].append(item['fields'])


In [ ]:
user_3_analysis = defaultdict(str)
user_4_analysis = defaultdict(str)
for item in users[3]:
    item['problems'] = '#'.join(sorted( [str(x) for x in item['problems']] ))
    user_3_analysis[item['prompt']] = item['problems']


for item in users[4]:
    item['problems'] = '#'.join(sorted( [str(x) for x in item['problems']] ))
    user_4_analysis[item['prompt']] = item['problems']

len(user_3_analysis), len(user_4_analysis)

In [ ]:
user_3 = []
user_4 = []
problem_wise = defaultdict(list)
for key in merged_analysis.keys():
    if key not in user_3_analysis:
        user_3_analysis[key] = ''
    if key not in user_4_analysis:
        user_4_analysis[key] = ''
    user_3.append(user_3_analysis[key])
    user_4.append(user_4_analysis[key])

    for i in range(1,17):
        if str(i) in user_3_analysis[key].split('#'):
            problem_wise[i].append(1)
        else:
            problem_wise[i].append(0)
            
        if str(i) in user_4_analysis[key].split('#'):
            problem_wise[i].append(1)
        else:
            problem_wise[i].append(0)


In [ ]:
from sklearn.metrics import cohen_kappa_score
cohen_kappa_score(user_3, user_4)

In [ ]:
for i in range(1,17):
    user_3 = []
    user_4 = []
    # print(problem_wise[i])
    for index, item in enumerate(problem_wise[i]):
        if index%2==0:
            user_3.append(item)
        else:
            user_4.append(item)
    print(sum(user_3), sum(user_4))
    print(f"Problem {i}: {cohen_kappa_score(user_3, user_4)}")